# import

In [ ]:
import tensorflow as tf
from tensorflow.python.client import device_lib
#from tensorflow.python.ops import rnn, rnn_cell
from tensorflow.contrib import rnn
import numpy as np
import tensorflow.contrib.slim as slim
from compute_mcc import *
#import scipy.io as sio
import math
import h5py
#from compute_mcc import compute_mcc,metrics,_fast_hist,label_accuracy_score
from hilbert import hilbertCurve
#from compute_IoU import compute_precision,bb_IoU
# sys.path.append('.')
import os,sys
from scipy import signal
import time
import skimage
import skimage.io, skimage.transform
from skimage.transform import resize
from skimage.util import view_as_windows
import scipy.misc
import scipy.io as sio
from skimage import img_as_uint
import matplotlib.pyplot as plt
# import pandas
import glob 
import datetime
from skimage.color import rgb2ycbcr 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import pywt
# import cv2
import re
from PIL import Image
import os
import bisect
import pickle

# Fileters

In [ ]:
SRM_Kernel = np.array([
    [[0,0,0,0,0],[0,0,0,0,0],[0,0,-1,1,0],[0,0,0,0,0],[0,0,0,0,0]],
    [[0,0,0,0,0],[0,0,0,0,0],[0,1,-1,0,0],[0,0,0,0,0],[0,0,0,0,0]],
    [[0,0,0,0,0],[0,0,1,0,0],[0,0,-1,0,0],[0,0,0,0,0],[0,0,0,0,0]],
    [[0,0,0,0,0],[0,0,0,0,0],[0,0,-1,0,0],[0,0,1,0,0],[0,0,0,0,0]],
    [[0,0,0,0,0],[0,0,0,0,0],[0,1,-2,1,0],[0,0,0,0,0],[0,0,0,0,0]],
    [[0,0,0,0,0],[0,-1,2,-1,0],[0,2,-4,2,0],[0,-1,2,-1,0],[0,0,0,0,0]],
    [[-1,2,-2,2,-1],[2,-6,8,-6,2],[-2,8,-12,8,-2],[2,-6,8,-6,2],[-1,2,-2,2,-1]],
])
SRM_Kernel = np.vstack((SRM_Kernel, SRM_Kernel, SRM_Kernel)).reshape(3, 7, 5, 5).transpose(2,3,0,1)

# parameters

In [ ]:
tf.reset_default_graph()
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
log_device_placement = True
# Parameters
lr = 0.00003
training_iters = 50000000
batch_size = 30
display_step = 10
nb_nontamp_img=16960
nb_tamp_img=68355
nbFilter=32


# LSTM network parameters
n_input = 240 # data input (img shape: 64x64)
n_steps = 64 # timesteps
nBlock=int(math.sqrt(n_steps))
n_hidden = 64# hidden layer num of features
nStride=int(math.sqrt(n_hidden))
# other parameters
imSize=256
# Network Parameters
n_classes = 2 # manipulated vs unmanipulated
mx=127.0

# tf Graph input
input_layer = tf.placeholder("float", [None, imSize,imSize,3])
y= tf.placeholder("float", [2,None, imSize,imSize])
freqFeat=tf.placeholder("float", [None, 248,248,3])
# freqFeat=tf.placeholder("float", [None, 64,240])
# freqFeat=tf.placeholder("float", [None, 256,256,3])
filter = tf.Variable(tf.random_normal([5,5,3,9]))
ratio=15.0 #tf.placeholder("float",[1])
#out_rnn=tf.placeholder("float", [None, 128,128,3])
# W1 = tf.get_variable('W1', [5,5,3, 10], tf.float32, xavier_initializer())
# b1 = tf.Variable(tf.random_normal([5,5,3]))

############################################################################
#total_layers = 25 #Specify how deep we want our network
units_between_stride = 2
upsample_factor=16
beta=.01
outSize=16
############################################################################
seq = np.linspace(0,63,64).astype(int)
order3 = hilbertCurve(3)
order3 = np.reshape(order3,(64))
print(seq)
print(order3)
hilbert_ind = np.lexsort((seq,order3))
actual_ind=np.lexsort((seq,hilbert_ind))

weights = {
    'out': tf.Variable(tf.random_normal([64,64,nbFilter]))
}
biases = {
    'out': tf.Variable(tf.random_normal([nbFilter]))
}

atrous_fil = tf.Variable(tf.random_normal([3, 3, 256,256]),name = 'atrous_fil')
atrous_fil_1 =tf.Variable(tf.random_normal([3, 3, 256,256]),name ='atrous_fil_1')
atrous_fil_2 =tf.Variable(tf.random_normal([3, 3, 256,256]),name ='atrous_fil_2')


atrous_fil1 = tf.Variable(tf.random_normal([3, 3, 256,256]),name ='atrous_fil1')
atrous_fil2 = tf.Variable(tf.random_normal([3, 3, 256,256]),name ='atrous_fil2')
atrous_fil3 = tf.Variable(tf.random_normal([3, 3, 256,256]),name ='atrous_fil3')

## base-model 

In [ ]:
with tf.device('/gpu:1'):

    def conv_mask_gt(z): 
        # Get ones for each class instead of a number -- we need that
        # for cross-entropy loss later on. Sometimes the groundtruth
        # masks have values other than 1 and 0. 
#         class_labels_tensor = (z==1)
#         background_labels_tensor = (z==0)
        
        class_labels_tensor = (z==1)
        background_labels_tensor = (z==0)
        # Convert the boolean values into floats -- so that
        # computations in cross-entropy loss is correct
        bit_mask_class = np.float32(class_labels_tensor)
        bit_mask_background = np.float32(background_labels_tensor)
        combined_mask=[]
        combined_mask.append(bit_mask_background)
        combined_mask.append(bit_mask_class)
        #combined_mask = tf.concat(concat_dim=3, values=[bit_mask_background,bit_mask_class])		

        # Lets reshape our input so that it becomes suitable for 
        # tf.softmax_cross_entropy_with_logits with [batch_size, num_classes]
        #flat_labels = tf.reshape(tensor=combined_mask, shape=(-1, 2))	
        return combined_mask#flat_labels

    def get_kernel_size(factor):
        #Find the kernel size given the desired factor of upsampling.
        return 2 * factor - factor % 2

    def upsample_filt(size):
        """
        Make a 2D bilinear kernel suitable for upsampling of the given (h, w) size.
        """
        factor = (size + 1) // 2
        if size % 2 == 1:
            center = factor - 1
        else:
            center = factor - 0.5
        og = np.ogrid[:size, :size]
        return (1 - abs(og[0] - center) / factor) * \
            (1 - abs(og[1] - center) / factor)

    def bilinear_upsample_weights(factor, number_of_classes):
        """
        Create weights matrix for transposed convolution with bilinear filter
        initialization.
        """    
        filter_size = get_kernel_size(factor)

        weights = np.zeros((filter_size,filter_size,number_of_classes,number_of_classes), dtype=np.float32)    
        upsample_kernel = upsample_filt(filter_size)    
        for i in range(number_of_classes):        
            weights[:, :, i, i] = upsample_kernel    
        return weights


    def resUnit(input_layer,i,nbF):
        with tf.variable_scope("res_unit"+str(i)):
        #input_layer=tf.reshape(input_layer,[-1,64,64,3])
            part1 = slim.batch_norm(input_layer,activation_fn=None)
            part2 = tf.nn.relu(part1)
            part3 = slim.conv2d(part2,nbF,[3,3],activation_fn=None)
            part4 = slim.batch_norm(part3,activation_fn=None)
            part5 = tf.nn.relu(part4)
            part6 = slim.conv2d(part5,nbF,[3,3],activation_fn=None)	
            output = input_layer + part6
            return output

    #tf.reset_default_graph()

    def segNet(input_layer,bSize,freqFeat,weights,biases):
        # layer1: resblock, input size(256,256)
        layer1 = slim.conv2d(input_layer,nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(0))
        layer1 =resUnit(layer1,1,nbFilter)
        layer1 = tf.nn.relu(layer1)
        layer2=slim.max_pool2d(layer1, [2, 2], scope='pool_'+str(1))
        # layer2: resblock, input size(128,128)   
        layer2 = slim.conv2d(layer2,2*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(1))
        layer2 =resUnit(layer2,2,2*nbFilter)
        layer2 = tf.nn.relu(layer2)
        layer3=slim.max_pool2d(layer2, [2, 2], scope='pool_'+str(2))
        # layer3: resblock, input size(64,64) 
        layer3 = slim.conv2d(layer3,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(2))
        layer3 =resUnit(layer3,3,4*nbFilter)
        layer3 = tf.nn.relu(layer3)
        layer4=slim.max_pool2d(layer3, [2, 2], scope='pool_'+str(3))
        # layer4: resblock, input size(32,32) 
        layer4 = slim.conv2d(layer4,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(3))
        layer4 =resUnit(layer4,4,8*nbFilter)
        layer4 = tf.nn.relu(layer4)
        layer4=slim.max_pool2d(layer4, [2, 2], scope='pool_'+str(4))
        # end of layer4: resblock, input size(16,16)

        # lstm network 
        patches=tf.transpose(freqFeat,[1,0,2])
        patches=tf.gather(patches,hilbert_ind)
        patches=tf.transpose(patches,[1,0,2])         
        # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
        xCell=tf.unstack(patches, n_steps, 1)
        # 2 stacked layers
        stacked_lstm_cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(rnn.BasicLSTMCell(n_hidden),output_keep_prob=0.9) for _ in range(2)] )
        out, state = rnn.static_rnn(stacked_lstm_cell, xCell, dtype=tf.float32)
        # organizing the lstm output
        out=tf.gather(out,actual_ind)
        # convert to lstm output (64,batchSize,nbFilter)
        lstm_out=tf.matmul(out,weights['out'])+biases['out']
        print('lstm_out3:',lstm_out.shape)
        lstm_out=tf.transpose(lstm_out,[1,0,2])
        # convert to size(batchSize, 8,8, nbFilter)
        lstm_out=tf.reshape(lstm_out,[bSize,8,8,nbFilter])
        # perform batch normalization and activiation
        lstm_out=slim.batch_norm(lstm_out,activation_fn=None)
        lstm_out=tf.nn.relu(lstm_out)
        print('lstm_out3:',lstm_out.shape)
        # upsample lstm output to (batchSize, 16,16, nbFilter)
        temp=tf.random_normal([bSize,outSize,outSize,nbFilter])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(2, nbFilter)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        lstm_out = tf.nn.conv2d_transpose(lstm_out, upsample_filter_tensor,output_shape=uShape1,strides=[1, 2, 2, 1])

        # reduce the filter size to nbFilter for layer4
        top = slim.conv2d(layer4,nbFilter,[1,1], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_top')
        top = tf.nn.relu(top)
        # concatenate both lstm features and image features
        joint_out=tf.concat([top,lstm_out],3)
        print('joint_out:',joint_out.shape)
        
        # perform upsampling (batchSize, 64,64, 2*nbFilter)
        temp=tf.random_normal([bSize,outSize*4,outSize*4,2*nbFilter])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(4, 2*nbFilter)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        upsampled_layer4 = tf.nn.conv2d_transpose(joint_out, upsample_filter_tensor,output_shape=uShape1,strides=[1, 4, 4, 1]) 	
        print('upsampled_layer4:',upsampled_layer4.shape)
        # reduce filter sizes	
        upsampled_layer4 = slim.conv2d(upsampled_layer4,2,[1,1], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_'+str(4))
        upsampled_layer4=slim.batch_norm(upsampled_layer4,activation_fn=None)
        upsampled_layer4=tf.nn.relu(upsampled_layer4)
        print('upsampled_layer4:',upsampled_layer4.shape)
        # upsampling to (batchSize, 256,256, nbClasses)
        temp=tf.random_normal([bSize,outSize*16,outSize*16,2])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(4,2)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        upsampled_layer5 = tf.nn.conv2d_transpose(upsampled_layer4, upsample_filter_tensor,output_shape=uShape1,strides=[1, 4, 4, 1]) 
        print('upsampled_layer5:',upsampled_layer5.shape)
        #upsampled_layer5=slim.batch_norm(upsampled_layer5,activation_fn=None)
        #upsampled_layer5 = slim.conv2d(upsampled_layer5,2,[3,3], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_'+str(5))
        #upsampled_layer5=tf.nn.relu(upsampled_layer5)


        return upsampled_layer5


    y1=tf.transpose(y,[1,2,3,0])
    upsampled_logits=segNet(input_layer,batch_size,freqFeat,weights,biases)


    flat_pred=tf.reshape(upsampled_logits,(-1,n_classes))
    flat_y=tf.reshape(y1,(-1,n_classes))

    #loss1=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=flat_pred,labels=flat_y))

    loss = tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(flat_y,flat_pred, 1.0))

    #all_weights  = tf.trainable_variables()
    #regLoss = tf.add_n([ tf.nn.l2_loss(v) for v in all_weights ]) * beta
    #loss = 0.75*loss1+loss2
    trainer = tf.train.AdamOptimizer(learning_rate=lr)
    update = trainer.minimize(loss)
    #update2 = trainer.minimize(loss2)

    probabilities=tf.nn.softmax(flat_pred)
    correct_pred=tf.equal(tf.argmax(probabilities,1),tf.argmax(flat_y,1))
    accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

    y_actual=tf.argmax(flat_y,1)
    y_pred=tf.argmax(flat_pred,1)

    mask_actual= tf.argmax(y1,3)
    mask_pred=tf.argmax(upsampled_logits,3)


# Initializing the variables
# init = tf.initialize_all_variables()

saver = tf.train.Saver()

config=tf.ConfigProto()
config.allow_soft_placement=True
config.log_device_placement=True
config.gpu_options.allow_growth=True
#config.gpu_options.per_process_gpu_memory_fraction = 0.4


## PL-GNet

In [ ]:
with tf.device('/gpu:1'):

    def conv_mask_gt(z): 
        # Get ones for each class instead of a number -- we need that
        # for cross-entropy loss later on. Sometimes the groundtruth
        # masks have values other than 1 and 0. 
#         class_labels_tensor = (z==1)
#         background_labels_tensor = (z==0)
        
        class_labels_tensor = (z==1)
        background_labels_tensor = (z==0)
        # Convert the boolean values into floats -- so that
        # computations in cross-entropy loss is correct
        bit_mask_class = np.float32(class_labels_tensor)
        bit_mask_background = np.float32(background_labels_tensor)
        combined_mask=[]
        combined_mask.append(bit_mask_background)
        combined_mask.append(bit_mask_class)
        #combined_mask = tf.concat(concat_dim=3, values=[bit_mask_background,bit_mask_class])		

        # Lets reshape our input so that it becomes suitable for 
        # tf.softmax_cross_entropy_with_logits with [batch_size, num_classes]
        #flat_labels = tf.reshape(tensor=combined_mask, shape=(-1, 2))	
        return combined_mask#flat_labels

    def get_kernel_size(factor):
        #Find the kernel size given the desired factor of upsampling.
        return 2 * factor - factor % 2

    def upsample_filt(size):
        """
        Make a 2D bilinear kernel suitable for upsampling of the given (h, w) size.
        """
        factor = (size + 1) // 2
        if size % 2 == 1:
            center = factor - 1
        else:
            center = factor - 0.5
        og = np.ogrid[:size, :size]
        return (1 - abs(og[0] - center) / factor) * \
            (1 - abs(og[1] - center) / factor)

    def bilinear_upsample_weights(factor, number_of_classes):
        """
        Create weights matrix for transposed convolution with bilinear filter
        initialization.
        """    
        filter_size = get_kernel_size(factor)

        weights = np.zeros((filter_size,filter_size,number_of_classes,number_of_classes), dtype=np.float32)    
        upsample_kernel = upsample_filt(filter_size)    
        for i in range(number_of_classes):        
            weights[:, :, i, i] = upsample_kernel    
        return weights


    def resUnit(input_layer,i,nbF):
        with tf.variable_scope("res_unit"+str(i)):
        #input_layer=tf.reshape(input_layer,[-1,64,64,3])
            part1 = slim.batch_norm(input_layer,activation_fn=None)
            part2 = tf.nn.relu(part1)
            part3 = slim.conv2d(part2,nbF,[3,3],activation_fn=None)
            part4 = slim.batch_norm(part3,activation_fn=None)
            part5 = tf.nn.relu(part4)
            part6 = slim.conv2d(part5,nbF,[3,3],activation_fn=None)	
            output = input_layer + part6
            return output

    #tf.reset_default_graph()

    def segNet(input_layer,bSize,freqFeat,weights,biases):
        
        # layer1: resblock, input size(256,256)
        layer1 = tf.nn.conv2d(input_layer, SRM_Kernel, strides=[1,1,1,1],padding ='SAME',name = 'SRM_out' )
        layer2 = tf.nn.conv2d(input_layer, filter, strides=[1,1,1,1],padding ='SAME',name = 'SRM_out1' )
        concat = tf.concat([layer1, layer2], axis=3, name='concat')
        print('concat:',concat.shape)
        
        Conv_1 = slim.conv2d(concat,nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(1))
        ReLu_1 = tf.nn.relu(Conv_1)
        Pool_1 = slim.max_pool2d(ReLu_1, [2, 2], scope='pool_'+str(1))
        print('Pool_1:',Pool_1.shape)
        
        # layer2: resblock, input size(128,128)   
        Conv_2 = slim.conv2d(Pool_1,2*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(2))
        ReLu_2 = tf.nn.relu(Conv_2)
        Conv_3 = slim.conv2d(ReLu_2,2*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(3))
        ReLu_3 = tf.nn.relu(Conv_3)
        Pool_2 = slim.max_pool2d(ReLu_3, [2, 2], scope='pool_'+str(2))
        print('Pool_2:',Pool_2.shape)
        
        # layer3: resblock, input size(64,64) 
        Conv_4 = slim.conv2d(Pool_2,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(4))
        ReLu_4 = tf.nn.relu(Conv_4)
        Conv_5 = slim.conv2d(ReLu_4,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(5))
        ReLu_5 = tf.nn.relu(Conv_5)
        Conv_6 = slim.conv2d(ReLu_5,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(6))
        ReLu_6 = tf.nn.relu(Conv_6)
        Pool_3 = slim.max_pool2d(ReLu_6, [2, 2], scope='pool_'+str(3))
        # layer4: resblock, input size(32,32) 
        print('Pool_3:',Pool_3.shape)
        
        Conv_7 = slim.conv2d(Pool_3,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(7))
        ReLu_7= tf.nn.relu(Conv_7)
        Conv_8 = slim.conv2d(ReLu_7,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(8))
        ReLu_8 = tf.nn.relu(Conv_8)
        Conv_9 = slim.conv2d(ReLu_8,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(9))
        ReLu_9 = tf.nn.relu(Conv_9)
        print('ReLu_9:',ReLu_9.shape)
        # layer4: resblock, input size(32,32) 
        
        Conv_11_AC = tf.nn.atrous_conv2d(ReLu_9,atrous_fil,rate =2,padding = 'SAME',name = 'conv_'+str(11))
        BN_2 = slim.batch_norm(Conv_11_AC,activation_fn=None)
        Tan_2 = tf.nn.relu(BN_2)
        Conv_12_AC = tf.nn.atrous_conv2d(Tan_2,atrous_fil_1,rate =2,padding = 'SAME',name = 'conv_'+str(12))
        BN_3 = slim.batch_norm(Conv_12_AC,activation_fn=None)
        Tan_3 = tf.nn.relu(BN_3)
        Conv_13_AC = tf.nn.atrous_conv2d(Tan_3,atrous_fil_2,rate =2,padding = 'SAME',name = 'conv_'+str(13))
        BN_4 = slim.batch_norm(Conv_13_AC,activation_fn=None)
        Tan_4 = tf.nn.relu(BN_4)
        output = ReLu_9 + Tan_4
#         output = slim.max_pool2d(output, [2, 2], scope='pool_'+str(4))
             
        layer6 = slim.conv2d(output,8*nbFilter,[1,1],normalizer_fn=slim.batch_norm,scope='conv_'+str(14))
        layer6 = tf.nn.relu(layer6)
       
                                      
        layer7 = tf.nn.atrous_conv2d(output,atrous_fil1,rate = 3,padding = 'SAME',name='conv_'+str(15))
        layer7 = tf.nn.relu(layer7)
        
                                      
        layer8 = tf.nn.atrous_conv2d(output,atrous_fil2,rate = 5,padding = 'SAME',name='conv_'+str(16))
        layer8 = tf.nn.relu(layer8)
        
                                      
        layer9 = tf.nn.atrous_conv2d(output,atrous_fil3,rate = 7,padding = 'SAME',name='conv_'+str(17))
        layer9 = tf.nn.relu(layer9)
       
        
        layer10 = layer6+layer7+layer8+layer9
        print('layer10:',layer10.shape)
        
        layer12 = tf.nn.relu(layer10)
        layer13 = slim.max_pool2d(layer12, [2, 2], scope='pool_'+str(4))
        print('layer13:',layer13.shape)
     
        # lstm network 
        layer55 = slim.conv2d(freqFeat,16,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(18))
        layer55 = tf.nn.relu(layer55)
        layer66 = slim.conv2d(layer55,1,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(19))
        layer66 = tf.nn.relu(layer66)
        print('layer6_shape:',layer66.shape)  # ( bs, 256, 256, 1)
        
        layer77=tf.transpose(layer66,[0,3,1,2])
        print('layer7_shape:',layer77.shape)  # ( bs, 256, 256, 1)
        
        y_list = tf.split(layer77,8,axis=3)
        print('y_list_shape2:',len(y_list))
        xy_list = [tf.split(x,8,axis =2) for x in y_list] ##
        print('xy_list_shape2:',len(xy_list))
        xy = [item for items in xy_list for item in items]
        print('xy_shape:',len(xy))
#         xy = torch.cat(xy,1)
        xy = tf.concat(xy,1)
        print('xy_shape2:',xy.shape)
#         patches = xy.view(-1,64,64)#
        patches = tf.reshape(xy,(-1,64,31*31))
        print('patches_shape2:',patches.shape)
        
        
#         patches=tf.transpose(freqFeat,[1,0,2])
#         patches=tf.gather(patches,hilbert_ind)
#         patches=tf.transpose(patches,[1,0,2])
#         print('patches:',patches.shape)
        # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
        xCell=tf.unstack(patches, n_steps, 1)
        # 2 stacked layers
        stacked_lstm_cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(rnn.BasicLSTMCell(n_hidden),output_keep_prob=0.9) for _ in range(2)] )
        out, state = rnn.static_rnn(stacked_lstm_cell, xCell, dtype=tf.float32)
        # organizing the lstm output
        out=tf.gather(out,actual_ind)
        # convert to lstm output (64,batchSize,nbFilter)
        lstm_out=tf.matmul(out,weights['out'])+biases['out']
        print('lstm_out1:',lstm_out.shape)
        lstm_out=tf.transpose(lstm_out,[1,0,2])
        print('lstm_out2:',lstm_out.shape)
        
        # convert to size(batchSize, 8,8, nbFilter)
        lstm_out=tf.reshape(lstm_out,[bSize,8,8,nbFilter])
        # perform batch normalization and activiation
        lstm_out=slim.batch_norm(lstm_out,activation_fn=None)
        lstm_out=tf.nn.relu(lstm_out)
        print('lstm_out3:',lstm_out.shape)
        # upsample lstm output to (batchSize, 16,16, nbFilter)
        temp=tf.random_normal([bSize,outSize,outSize,nbFilter])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(2, nbFilter)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        lstm_out = tf.nn.conv2d_transpose(lstm_out, upsample_filter_tensor,output_shape=uShape1,strides=[1, 2, 2, 1])
        print('lstm_out4:',lstm_out.shape)
        # reduce the filter size to nbFilter for layer4
        top = slim.conv2d(layer13,nbFilter,[1,1], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_top')
        top = tf.nn.relu(top)
        print('top:',top.shape)
        # concatenate both lstm features and image features
        joint_out=tf.concat([top,lstm_out],3)
        print('joint_out:',joint_out.shape)
        # perform upsampling (batchSize, 64,64, 2*nbFilter)
        temp=tf.random_normal([bSize,outSize*4,outSize*4,2*nbFilter])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(4, 2*nbFilter)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        upsampled_layer4 = tf.nn.conv2d_transpose(joint_out, upsample_filter_tensor,output_shape=uShape1,strides=[1, 4, 4, 1]) 	
        # reduce filter sizes	
        upsampled_layer4 = slim.conv2d(upsampled_layer4,2,[1,1], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_'+str(12))
        upsampled_layer4=slim.batch_norm(upsampled_layer4,activation_fn=None)
        upsampled_layer4=tf.nn.relu(upsampled_layer4)
        # upsampling to (batchSize, 256,256, nbClasses)
        temp=tf.random_normal([bSize,outSize*16,outSize*16,2])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(4,2)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        upsampled_layer5 = tf.nn.conv2d_transpose(upsampled_layer4, upsample_filter_tensor,output_shape=uShape1,strides=[1, 4, 4, 1]) 
        #upsampled_layer5=slim.batch_norm(upsampled_layer5,activation_fn=None)
        #upsampled_layer5 = slim.conv2d(upsampled_layer5,2,[3,3], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_'+str(5))
        #upsampled_layer5=tf.nn.relu(upsampled_layer5)


        return upsampled_layer5


    y1=tf.transpose(y,[1,2,3,0])
    upsampled_logits=segNet(input_layer,batch_size,freqFeat,weights,biases)
    print('upsampled_logits_shape:',upsampled_logits.shape)

    flat_pred=tf.reshape(upsampled_logits,(-1,n_classes))
    print('flat_pred_shape:',flat_pred.shape)
    
    flat_y=tf.reshape(y1,(-1,n_classes))

    #loss1=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=flat_pred,labels=flat_y))

    loss = tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(flat_y,flat_pred, 1.0))

    #all_weights  = tf.trainable_variables()
    #regLoss = tf.add_n([ tf.nn.l2_loss(v) for v in all_weights ]) * beta
    #loss = 0.75*loss1+loss2
    trainer = tf.train.AdamOptimizer(learning_rate=lr)
    update = trainer.minimize(loss)
    #update2 = trainer.minimize(loss2)

    probabilities=tf.nn.softmax(flat_pred)
    correct_pred=tf.equal(tf.argmax(probabilities,1),tf.argmax(flat_y,1))
    accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

    y_actual=tf.argmax(flat_y,1)
    y_pred=tf.argmax(flat_pred,1)

    mask_actual= tf.argmax(y1,3)
    mask_pred=tf.argmax(upsampled_logits,3)


# Initializing the variables
# init = tf.initialize_all_variables()

saver = tf.train.Saver()

config=tf.ConfigProto()
config.allow_soft_placement=True
config.log_device_placement=True
config.gpu_options.allow_growth=True
#config.gpu_options.per_process_gpu_memory_fraction = 0.4

### PL-GNet_no_LSTM

In [ ]:
with tf.device('/gpu:1'):

    def conv_mask_gt(z): 
        # Get ones for each class instead of a number -- we need that
        # for cross-entropy loss later on. Sometimes the groundtruth
        # masks have values other than 1 and 0. 
#         class_labels_tensor = (z==1)
#         background_labels_tensor = (z==0)
        
        class_labels_tensor = (z==1)
        background_labels_tensor = (z==0)
        # Convert the boolean values into floats -- so that
        # computations in cross-entropy loss is correct
        bit_mask_class = np.float32(class_labels_tensor)
        bit_mask_background = np.float32(background_labels_tensor)
        combined_mask=[]
        combined_mask.append(bit_mask_background)
        combined_mask.append(bit_mask_class)
        #combined_mask = tf.concat(concat_dim=3, values=[bit_mask_background,bit_mask_class])		

        # Lets reshape our input so that it becomes suitable for 
        # tf.softmax_cross_entropy_with_logits with [batch_size, num_classes]
        #flat_labels = tf.reshape(tensor=combined_mask, shape=(-1, 2))	
        return combined_mask#flat_labels

    def get_kernel_size(factor):
        #Find the kernel size given the desired factor of upsampling.
        return 2 * factor - factor % 2

    def upsample_filt(size):
        """
        Make a 2D bilinear kernel suitable for upsampling of the given (h, w) size.
        """
        factor = (size + 1) // 2
        if size % 2 == 1:
            center = factor - 1
        else:
            center = factor - 0.5
        og = np.ogrid[:size, :size]
        return (1 - abs(og[0] - center) / factor) * \
            (1 - abs(og[1] - center) / factor)

    def bilinear_upsample_weights(factor, number_of_classes):
        """
        Create weights matrix for transposed convolution with bilinear filter
        initialization.
        """    
        filter_size = get_kernel_size(factor)

        weights = np.zeros((filter_size,filter_size,number_of_classes,number_of_classes), dtype=np.float32)    
        upsample_kernel = upsample_filt(filter_size)    
        for i in range(number_of_classes):        
            weights[:, :, i, i] = upsample_kernel    
        return weights


    def resUnit(input_layer,i,nbF):
        with tf.variable_scope("res_unit"+str(i)):
        #input_layer=tf.reshape(input_layer,[-1,64,64,3])
            part1 = slim.batch_norm(input_layer,activation_fn=None)
            part2 = tf.nn.relu(part1)
            part3 = slim.conv2d(part2,nbF,[3,3],activation_fn=None)
            part4 = slim.batch_norm(part3,activation_fn=None)
            part5 = tf.nn.relu(part4)
            part6 = slim.conv2d(part5,nbF,[3,3],activation_fn=None)	
            output = input_layer + part6
            return output

    #tf.reset_default_graph()

    def segNet(input_layer,bSize,freqFeat,weights,biases):
        
        # layer1: resblock, input size(256,256)
        layer1 = tf.nn.conv2d(input_layer, Bayar_Kernel, strides=[1,1,1,1],padding ='SAME',name = 'SRM_out' )
        layer2 = tf.nn.conv2d(input_layer, filter, strides=[1,1,1,1],padding ='SAME',name = 'SRM_out1' )
        layer3 = tf.nn.conv2d(input_layer, SRM_Kernel, strides=[1,1,1,1],padding ='SAME',name = 'SRM_out1' )
        concat = tf.concat([layer1, layer2,layer3], axis=3, name='concat')
        print('concat:',concat.shape)
        
        Conv_1 = slim.conv2d(concat,nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(1))
        ReLu_1 = tf.nn.relu(Conv_1)
        Pool_1 = slim.max_pool2d(ReLu_1, [2, 2], scope='pool_'+str(1))
        print('Pool_1:',Pool_1.shape)
        
        # layer2: resblock, input size(128,128)   
        Conv_2 = slim.conv2d(Pool_1,2*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(2))
        ReLu_2 = tf.nn.relu(Conv_2)
        Conv_3 = slim.conv2d(ReLu_2,2*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(3))
        ReLu_3 = tf.nn.relu(Conv_3)
        Pool_2 = slim.max_pool2d(ReLu_3, [2, 2], scope='pool_'+str(2))
        print('Pool_2:',Pool_2.shape)
        
        # layer3: resblock, input size(64,64) 
        Conv_4 = slim.conv2d(Pool_2,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(4))
        ReLu_4 = tf.nn.relu(Conv_4)
        Conv_5 = slim.conv2d(ReLu_4,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(5))
        ReLu_5 = tf.nn.relu(Conv_5)
        Conv_6 = slim.conv2d(ReLu_5,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(6))
        ReLu_6 = tf.nn.relu(Conv_6)
        Pool_3 = slim.max_pool2d(ReLu_6, [2, 2], scope='pool_'+str(3))
        # layer4: resblock, input size(32,32) 
        print('Pool_3:',Pool_3.shape)
        
        Conv_7 = slim.conv2d(Pool_3,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(7))
        ReLu_7= tf.nn.relu(Conv_7)
        Conv_8 = slim.conv2d(ReLu_7,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(8))
        ReLu_8 = tf.nn.relu(Conv_8)
        Conv_9 = slim.conv2d(ReLu_8,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(9))
        ReLu_9 = tf.nn.relu(Conv_9)
        print('ReLu_9:',ReLu_9.shape)
        # layer4: resblock, input size(32,32) 
#         print('Pool_4:',Pool_4.shape)
        
#         Conv_10_AC = slim.conv2d(ReLu_9,8*nbFilter,[1,1],normalizer_fn=slim.batch_norm,scope='conv_'+str(10))
#         Tan_1 = tf.nn.relu(Conv_10_AC)
        
        Conv_11_AC = tf.nn.atrous_conv2d(ReLu_9,atrous_fil,rate =2,padding = 'SAME',name = 'conv_'+str(11))
        BN_2 = slim.batch_norm(Conv_11_AC,activation_fn=None)
        Tan_2 = tf.nn.relu(BN_2)
        Conv_12_AC = tf.nn.atrous_conv2d(Tan_2,atrous_fil_1,rate =2,padding = 'SAME',name = 'conv_'+str(12))
        BN_3 = slim.batch_norm(Conv_12_AC,activation_fn=None)
        Tan_3 = tf.nn.relu(BN_3)
        Conv_13_AC = tf.nn.atrous_conv2d(Tan_3,atrous_fil_2,rate =2,padding = 'SAME',name = 'conv_'+str(13))
        BN_4 = slim.batch_norm(Conv_13_AC,activation_fn=None)
        Tan_4 = tf.nn.relu(BN_4)
        output = ReLu_9 + Tan_4
#         output = slim.max_pool2d(output, [2, 2], scope='pool_'+str(4))
        
#         layer55 = tf.nn.relu(output)
       
        layer6 = slim.conv2d(output,8*nbFilter,[1,1],normalizer_fn=slim.batch_norm,scope='conv_'+str(14))
        layer6 = tf.nn.relu(layer6)
       
                                      
        layer7 = tf.nn.atrous_conv2d(output,atrous_fil1,rate = 3,padding = 'SAME',name='conv_'+str(15))
        layer7 = tf.nn.relu(layer7)
        
                                      
        layer8 = tf.nn.atrous_conv2d(output,atrous_fil2,rate = 5,padding = 'SAME',name='conv_'+str(16))
        layer8 = tf.nn.relu(layer8)
        
                                      
        layer9 = tf.nn.atrous_conv2d(output,atrous_fil3,rate = 7,padding = 'SAME',name='conv_'+str(17))
        layer9 = tf.nn.relu(layer9)
       
        
        layer10 = layer6+layer7+layer8+layer9
        print('layer10:',layer10.shape)
        
        layer12 = tf.nn.relu(layer10)
        layer13 = slim.max_pool2d(layer12, [2, 2], scope='pool_'+str(4))
        print('layer13:',layer13.shape)
     
#         # lstm network 
#         layer55 = slim.conv2d(freqFeat,16,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(18))
#         layer55 = tf.nn.relu(layer55)
#         layer66 = slim.conv2d(layer55,1,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(19))
#         layer66 = tf.nn.relu(layer66)
#         print('layer6_shape:',layer66.shape)  # ( bs, 256, 256, 1)
        
#         layer77=tf.transpose(layer66,[0,3,1,2])
#         print('layer7_shape:',layer77.shape)  # ( bs, 256, 256, 1)
        
#         y_list = tf.split(layer77,8,axis=3)
#         print('y_list_shape2:',len(y_list))
#         xy_list = [tf.split(x,8,axis =2) for x in y_list] ##
#         print('xy_list_shape2:',len(xy_list))
#         xy = [item for items in xy_list for item in items]
#         print('xy_shape:',len(xy))
# #         xy = torch.cat(xy,1)
#         xy = tf.concat(xy,1)
#         print('xy_shape2:',xy.shape)
# #         patches = xy.view(-1,64,64)#
#         patches = tf.reshape(xy,(-1,64,31*31))
#         print('patches_shape2:',patches.shape)
        
        
# #         patches=tf.transpose(freqFeat,[1,0,2])
# #         patches=tf.gather(patches,hilbert_ind)
# #         patches=tf.transpose(patches,[1,0,2])
# #         print('patches:',patches.shape)
#         # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
#         xCell=tf.unstack(patches, n_steps, 1)
#         # 2 stacked layers
#         stacked_lstm_cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(rnn.BasicLSTMCell(n_hidden),output_keep_prob=0.9) for _ in range(2)] )
#         out, state = rnn.static_rnn(stacked_lstm_cell, xCell, dtype=tf.float32)
#         # organizing the lstm output
#         out=tf.gather(out,actual_ind)
#         # convert to lstm output (64,batchSize,nbFilter)
#         lstm_out=tf.matmul(out,weights['out'])+biases['out']
#         print('lstm_out1:',lstm_out.shape)
#         lstm_out=tf.transpose(lstm_out,[1,0,2])
#         print('lstm_out2:',lstm_out.shape)
        
#         # convert to size(batchSize, 8,8, nbFilter)
#         lstm_out=tf.reshape(lstm_out,[bSize,8,8,nbFilter])
#         # perform batch normalization and activiation
#         lstm_out=slim.batch_norm(lstm_out,activation_fn=None)
#         lstm_out=tf.nn.relu(lstm_out)
#         print('lstm_out3:',lstm_out.shape)
#         # upsample lstm output to (batchSize, 16,16, nbFilter)
#         temp=tf.random_normal([bSize,outSize,outSize,nbFilter])
#         uShape1=tf.shape(temp)
#         upsample_filter_np = bilinear_upsample_weights(2, nbFilter)
#         upsample_filter_tensor = tf.constant(upsample_filter_np)
#         lstm_out = tf.nn.conv2d_transpose(lstm_out, upsample_filter_tensor,output_shape=uShape1,strides=[1, 2, 2, 1])
#         print('lstm_out4:',lstm_out.shape)
        # reduce the filter size to nbFilter for layer4
        top = slim.conv2d(layer13,2*nbFilter,[1,1], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_top')
        top = tf.nn.relu(top)
        print('top:',top.shape)
        # concatenate both lstm features and image features
#         joint_out=tf.concat([top,lstm_out],3)
#         print('joint_out:',joint_out.shape)
        # perform upsampling (batchSize, 64,64, 2*nbFilter)
        temp=tf.random_normal([bSize,outSize*4,outSize*4,2*nbFilter])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(4, 2*nbFilter)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        upsampled_layer4 = tf.nn.conv2d_transpose(top, upsample_filter_tensor,output_shape=uShape1,strides=[1, 4, 4, 1]) 	
        # reduce filter sizes	
        upsampled_layer4 = slim.conv2d(upsampled_layer4,2,[1,1], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_'+str(12))
        upsampled_layer4=slim.batch_norm(upsampled_layer4,activation_fn=None)
        upsampled_layer4=tf.nn.relu(upsampled_layer4)
        # upsampling to (batchSize, 256,256, nbClasses)
        temp=tf.random_normal([bSize,outSize*16,outSize*16,2])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(4,2)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        upsampled_layer5 = tf.nn.conv2d_transpose(upsampled_layer4, upsample_filter_tensor,output_shape=uShape1,strides=[1, 4, 4, 1]) 
        #upsampled_layer5=slim.batch_norm(upsampled_layer5,activation_fn=None)
        #upsampled_layer5 = slim.conv2d(upsampled_layer5,2,[3,3], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_'+str(5))
        #upsampled_layer5=tf.nn.relu(upsampled_layer5)


        return upsampled_layer5


    y1=tf.transpose(y,[1,2,3,0])
    upsampled_logits=segNet(input_layer,batch_size,freqFeat,weights,biases)
    print('upsampled_logits_shape:',upsampled_logits.shape)

    flat_pred=tf.reshape(upsampled_logits,(-1,n_classes))
    print('flat_pred_shape:',flat_pred.shape)
    
    flat_y=tf.reshape(y1,(-1,n_classes))

    #loss1=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=flat_pred,labels=flat_y))

    loss = tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(flat_y,flat_pred, 1.0))

    #all_weights  = tf.trainable_variables()
    #regLoss = tf.add_n([ tf.nn.l2_loss(v) for v in all_weights ]) * beta
    #loss = 0.75*loss1+loss2
    trainer = tf.train.AdamOptimizer(learning_rate=lr)
    update = trainer.minimize(loss)
    #update2 = trainer.minimize(loss2)

    probabilities=tf.nn.softmax(flat_pred)
    correct_pred=tf.equal(tf.argmax(probabilities,1),tf.argmax(flat_y,1))
    accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

    y_actual=tf.argmax(flat_y,1)
    y_pred=tf.argmax(flat_pred,1)

    mask_actual= tf.argmax(y1,3)
    mask_pred=tf.argmax(upsampled_logits,3)


# Initializing the variables
# init = tf.initialize_all_variables()

saver = tf.train.Saver()

config=tf.ConfigProto()
config.allow_soft_placement=True
config.log_device_placement=True
config.gpu_options.allow_growth=True
#config.gpu_options.per_process_gpu_memory_fraction = 0.4


### PL-GNet+original-feat

In [ ]:
with tf.device('/gpu:1'):

    def conv_mask_gt(z): 
        # Get ones for each class instead of a number -- we need that
        # for cross-entropy loss later on. Sometimes the groundtruth
        # masks have values other than 1 and 0. 
#         class_labels_tensor = (z==1)
#         background_labels_tensor = (z==0)
        
        class_labels_tensor = (z==1)
        background_labels_tensor = (z==0)
        # Convert the boolean values into floats -- so that
        # computations in cross-entropy loss is correct
        bit_mask_class = np.float32(class_labels_tensor)
        bit_mask_background = np.float32(background_labels_tensor)
        combined_mask=[]
        combined_mask.append(bit_mask_background)
        combined_mask.append(bit_mask_class)
        #combined_mask = tf.concat(concat_dim=3, values=[bit_mask_background,bit_mask_class])		

        # Lets reshape our input so that it becomes suitable for 
        # tf.softmax_cross_entropy_with_logits with [batch_size, num_classes]
        #flat_labels = tf.reshape(tensor=combined_mask, shape=(-1, 2))	
        return combined_mask#flat_labels

    def get_kernel_size(factor):
        #Find the kernel size given the desired factor of upsampling.
        return 2 * factor - factor % 2

    def upsample_filt(size):
        """
        Make a 2D bilinear kernel suitable for upsampling of the given (h, w) size.
        """
        factor = (size + 1) // 2
        if size % 2 == 1:
            center = factor - 1
        else:
            center = factor - 0.5
        og = np.ogrid[:size, :size]
        return (1 - abs(og[0] - center) / factor) * \
            (1 - abs(og[1] - center) / factor)

    def bilinear_upsample_weights(factor, number_of_classes):
        """
        Create weights matrix for transposed convolution with bilinear filter
        initialization.
        """    
        filter_size = get_kernel_size(factor)

        weights = np.zeros((filter_size,filter_size,number_of_classes,number_of_classes), dtype=np.float32)    
        upsample_kernel = upsample_filt(filter_size)    
        for i in range(number_of_classes):        
            weights[:, :, i, i] = upsample_kernel    
        return weights


    def resUnit(input_layer,i,nbF):
        with tf.variable_scope("res_unit"+str(i)):
        #input_layer=tf.reshape(input_layer,[-1,64,64,3])
            part1 = slim.batch_norm(input_layer,activation_fn=None)
            part2 = tf.nn.relu(part1)
            part3 = slim.conv2d(part2,nbF,[3,3],activation_fn=None)
            part4 = slim.batch_norm(part3,activation_fn=None)
            part5 = tf.nn.relu(part4)
            part6 = slim.conv2d(part5,nbF,[3,3],activation_fn=None)	
            output = input_layer + part6
            return output

    #tf.reset_default_graph()

    def segNet(input_layer,bSize,freqFeat,weights,biases):
        
        # layer1: resblock, input size(256,256)
        layer1 = tf.nn.conv2d(input_layer, SRM_Kernel, strides=[1,1,1,1],padding ='SAME',name = 'SRM_out' )
        layer2 = tf.nn.conv2d(input_layer, filter, strides=[1,1,1,1],padding ='SAME',name = 'SRM_out1' )
        concat = tf.concat([layer1, layer2], axis=3, name='concat')
        print('concat:',concat.shape)
        
        Conv_1 = slim.conv2d(concat,nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(1))
        ReLu_1 = tf.nn.relu(Conv_1)
        Pool_1 = slim.max_pool2d(ReLu_1, [2, 2], scope='pool_'+str(1))
        print('Pool_1:',Pool_1.shape)
        
        # layer2: resblock, input size(128,128)   
        Conv_2 = slim.conv2d(Pool_1,2*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(2))
        ReLu_2 = tf.nn.relu(Conv_2)
        Conv_3 = slim.conv2d(ReLu_2,2*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(3))
        ReLu_3 = tf.nn.relu(Conv_3)
        Pool_2 = slim.max_pool2d(ReLu_3, [2, 2], scope='pool_'+str(2))
        print('Pool_2:',Pool_2.shape)
        
        # layer3: resblock, input size(64,64) 
        Conv_4 = slim.conv2d(Pool_2,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(4))
        ReLu_4 = tf.nn.relu(Conv_4)
        Conv_5 = slim.conv2d(ReLu_4,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(5))
        ReLu_5 = tf.nn.relu(Conv_5)
        Conv_6 = slim.conv2d(ReLu_5,4*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(6))
        ReLu_6 = tf.nn.relu(Conv_6)
        Pool_3 = slim.max_pool2d(ReLu_6, [2, 2], scope='pool_'+str(3))
        # layer4: resblock, input size(32,32) 
        print('Pool_3:',Pool_3.shape)
        
        Conv_7 = slim.conv2d(Pool_3,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(7))
        ReLu_7= tf.nn.relu(Conv_7)
        Conv_8 = slim.conv2d(ReLu_7,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(8))
        ReLu_8 = tf.nn.relu(Conv_8)
        Conv_9 = slim.conv2d(ReLu_8,8*nbFilter,[3,3],normalizer_fn=slim.batch_norm,scope='conv_'+str(9))
        ReLu_9 = tf.nn.relu(Conv_9)
        
        # layer4: resblock, input size(32,32) 
#         print('Pool_4:',Pool_4.shape)
        
#         Conv_10_AC = slim.conv2d(ReLu_9,8*nbFilter,[1,1],normalizer_fn=slim.batch_norm,scope='conv_'+str(10))
#         Tan_1 = tf.nn.relu(Conv_10_AC)
        
        Conv_11_AC = tf.nn.atrous_conv2d(ReLu_9,atrous_fil,rate =2,padding = 'SAME',name = 'conv_'+str(11))
        BN_2 = slim.batch_norm(Conv_11_AC,activation_fn=None)
        Tan_2 = tf.nn.relu(BN_2)
        Conv_12_AC = tf.nn.atrous_conv2d(Tan_2,atrous_fil_1,rate =2,padding = 'SAME',name = 'conv_'+str(12))
        BN_3 = slim.batch_norm(Conv_12_AC,activation_fn=None)
        Tan_3 = tf.nn.relu(BN_3)
        Conv_13_AC = tf.nn.atrous_conv2d(Tan_3,atrous_fil_2,rate =2,padding = 'SAME',name = 'conv_'+str(13))
        BN_4 = slim.batch_norm(Conv_13_AC,activation_fn=None)
        Tan_4 = tf.nn.relu(BN_4)
        output = ReLu_9 + Tan_4
#         output = slim.max_pool2d(output, [2, 2], scope='pool_'+str(4))
        
#         layer55 = tf.nn.relu(output)
       
        layer6 = slim.conv2d(output,8*nbFilter,[1,1],normalizer_fn=slim.batch_norm,scope='conv_'+str(14))
        layer6 = tf.nn.relu(layer6)
       
                                      
        layer7 = tf.nn.atrous_conv2d(output,atrous_fil1,rate = 3,padding = 'SAME',name='conv_'+str(15))
        layer7 = tf.nn.relu(layer7)
        
                                      
        layer8 = tf.nn.atrous_conv2d(output,atrous_fil2,rate = 5,padding = 'SAME',name='conv_'+str(16))
        layer8 = tf.nn.relu(layer8)
        
                                      
        layer9 = tf.nn.atrous_conv2d(output,atrous_fil3,rate = 7,padding = 'SAME',name='conv_'+str(17))
        layer9 = tf.nn.relu(layer9)
       
        
        layer10 = layer6+layer7+layer8+layer9
        print('layer10:',layer10.shape)
        
        layer12 = tf.nn.relu(layer10)
        layer13 = slim.max_pool2d(layer12, [2, 2], scope='pool_'+str(4))
        print('layer13:',layer13.shape)
     
        # lstm network 
        
         # lstm network 
        patches=tf.transpose(freqFeat,[1,0,2])
        patches=tf.gather(patches,hilbert_ind)
        patches=tf.transpose(patches,[1,0,2])         
        # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
        xCell=tf.unstack(patches, n_steps, 1)
        # 2 stacked layers
        stacked_lstm_cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(rnn.BasicLSTMCell(n_hidden),output_keep_prob=0.9) for _ in range(2)] )
        out, state = rnn.static_rnn(stacked_lstm_cell, xCell, dtype=tf.float32)
        # organizing the lstm output
        out=tf.gather(out,actual_ind)
        # convert to lstm output (64,batchSize,nbFilter)
        lstm_out=tf.matmul(out,weights['out'])+biases['out']
        lstm_out=tf.transpose(lstm_out,[1,0,2])
        # convert to size(batchSize, 8,8, nbFilter)
        lstm_out=tf.reshape(lstm_out,[bSize,8,8,nbFilter])
        # perform batch normalization and activiation
        lstm_out=slim.batch_norm(lstm_out,activation_fn=None)
        lstm_out=tf.nn.relu(lstm_out)
        # upsample lstm output to (batchSize, 16,16, nbFilter)
        temp=tf.random_normal([bSize,outSize,outSize,nbFilter])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(2, nbFilter)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        lstm_out = tf.nn.conv2d_transpose(lstm_out, upsample_filter_tensor,output_shape=uShape1,strides=[1, 2, 2, 1])
        print('lstm_out:',lstm_out.shape)
        # reduce the filter size to nbFilter for layer4
        top = slim.conv2d(layer13,nbFilter,[1,1], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_top')
        top = tf.nn.relu(top)
        print('top:',top.shape)
        # concatenate both lstm features and image features
        joint_out=tf.concat([top,lstm_out],3)
        # perform upsampling (batchSize, 64,64, 2*nbFilter)
        temp=tf.random_normal([bSize,outSize*4,outSize*4,2*nbFilter])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(4, 2*nbFilter)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        upsampled_layer4 = tf.nn.conv2d_transpose(joint_out, upsample_filter_tensor,output_shape=uShape1,strides=[1, 4, 4, 1]) 	
        # reduce filter sizes	
        upsampled_layer4 = slim.conv2d(upsampled_layer4,2,[1,1], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_'+str(12))
        upsampled_layer4=slim.batch_norm(upsampled_layer4,activation_fn=None)
        upsampled_layer4=tf.nn.relu(upsampled_layer4)
        # upsampling to (batchSize, 256,256, nbClasses)
        temp=tf.random_normal([bSize,outSize*16,outSize*16,2])
        uShape1=tf.shape(temp)
        upsample_filter_np = bilinear_upsample_weights(4,2)
        upsample_filter_tensor = tf.constant(upsample_filter_np)
        upsampled_layer5 = tf.nn.conv2d_transpose(upsampled_layer4, upsample_filter_tensor,output_shape=uShape1,strides=[1, 4, 4, 1]) 
        #upsampled_layer5=slim.batch_norm(upsampled_layer5,activation_fn=None)
        #upsampled_layer5 = slim.conv2d(upsampled_layer5,2,[3,3], normalizer_fn=slim.batch_norm, activation_fn=None, scope='conv_'+str(5))
        #upsampled_layer5=tf.nn.relu(upsampled_layer5)


        return upsampled_layer5


    y1=tf.transpose(y,[1,2,3,0])
    upsampled_logits=segNet(input_layer,batch_size,freqFeat,weights,biases)


    flat_pred=tf.reshape(upsampled_logits,(-1,n_classes))
    flat_y=tf.reshape(y1,(-1,n_classes))

    #loss1=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=flat_pred,labels=flat_y))

    loss = tf.reduce_mean(tf.nn.weighted_cross_entropy_with_logits(flat_y,flat_pred, 1.0))

    #all_weights  = tf.trainable_variables()
    #regLoss = tf.add_n([ tf.nn.l2_loss(v) for v in all_weights ]) * beta
    #loss = 0.75*loss1+loss2
    trainer = tf.train.AdamOptimizer(learning_rate=lr)
    update = trainer.minimize(loss)
    #update2 = trainer.minimize(loss2)

    probabilities=tf.nn.softmax(flat_pred)
    correct_pred=tf.equal(tf.argmax(probabilities,1),tf.argmax(flat_y,1))
    accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

    y_actual=tf.argmax(flat_y,1)
    y_pred=tf.argmax(flat_pred,1)

    mask_actual= tf.argmax(y1,3)
    mask_pred=tf.argmax(upsampled_logits,3)


# Initializing the variables
# init = tf.initialize_all_variables()

saver = tf.train.Saver()

config=tf.ConfigProto()
config.allow_soft_placement=True
config.log_device_placement=True
config.gpu_options.allow_growth=True
#config.gpu_options.per_process_gpu_memory_fraction = 0.4


# load data

In [ ]:
# loading Synthesized data
# freq1 = np.load('../dataset_npy/synthesized_imgs/Synthesized_imgS_feat.npy')

Img = np.load('../dataset_npy/synthesized_imgs/Synthesized_imgS.npy')
print('Img.....OK')
Lab = np.load('../dataset_npy/synthesized_imgs/Synthesized_labelS.npy')
print('Lab....OK')
freq1 = np.load('../dataset_npy/synthesized_imgs/Synthesized_imgS_cohere_feat.npy')
print('freq1...OK')


# loading NC_16 data for pretrain
Img_NC16 = np.load('../dataset_npy/NC_16/train/NC16_train_img.npy')
Img_NC16_img=np.shape(Img_NC16)
print ('Img_NC16_img.shape=',Img_NC16_img)
Lab_NC16 = np.load('../dataset_npy/NC_16/train/NC16_train_label.npy')
# feat_nc16 = np.load('../dataset_npy/NC_16/train/NC16_train_img_feat.npy')
feat_nc16 = np.load('../dataset_npy/NC_16/train/NC16_train_img_cohere_feat.npy')

vx=Img_NC16[-40:]
vx1=feat_nc16[-40:]
vy=Lab_NC16[-40:]


Img_NC16 = Img_NC16[:-40]
Lab_NC16 = Lab_NC16[:-40]
feat_nc16 = feat_nc16[:-40]
print('Img_NC16.shape=',Img_NC16.shape)

tx = np.load('../dataset_npy/NC_16/test/NC16_test_img.npy')
ty = np.load('../dataset_npy/NC_16/test/NC16_test_label.npy')
# freq4 = np.load('../dataset_npy/NC_16/test/NC16_test_img_feat.npy')
freq4 = np.load('../dataset_npy/NC_16/test/NC16_test_img_cohere_feat.npy')
print('freq4.shape=',freq4.shape)

## function

In [ ]:
def next_batch(size, batch_size, batch_ratio=1.0):
    import random
    rand = list(range(size))
    while True:
        random.shuffle(rand)
        for i in range(int(batch_ratio * size // batch_size)):
            yield rand[i*batch_size:(i+1)*batch_size]

In [ ]:
def compute_pos_neg(y_actual, y_hat):
    TP = 0; FP = 0;TN = 0; FN = 0
#     print('y_hat_shape',y_hat.shape)
#     print('y_actual_shape',y_actual.shape)
#     print('y_hat',y_hat)
#     print('y_actual',y_actual)
    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
            TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
            FP += 1
        if y_actual[i]==y_hat[i]==0:
            TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
            FN += 1

    return TP,FP,TN,FN

In [ ]:
def metrics(TP,FP,TN,FN):
    a=TP+FP
    b=TP+FN
    c=TN+FP
    d=TN+FN
    mcc=((TP*TN)-(FP*FN))/(math.sqrt(float(a*b*c*d)+0.0001))
    F1=(2*TP)/float(2*TP+FP+FN+.0000001)
    precision=TP/float(TP+FP+.0000001)
    #recall=TP/float(TP+FN+.0000001)
    return precision


In [ ]:
show_freq =1
learn_step =0
learn_rate = 0.00003
train_step = 100000
batch_size = 32
with tf.Session(config=config) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    tnb = next_batch(Lab.shape[0], batch_size)
    vnb = next_batch(vx.shape[0], batch_size)
    total_epoch = math.ceil(train_step / (Lab.shape[0] // batch_size))
    print('total_epoch=',total_epoch)
    
    while True:
        
        train_loss = 0
        train_batches = int(show_freq * Lab.shape[0] // batch_size)
        print('train_batches=',train_batches)
        
        for i in range(train_batches):
#             print('\ri: %d'%i)
            sys.stdout.write('\r' + str(i))
            sys.stdout.flush()
#             print('\ri: %d'%i,  end= flush=True)
            select = next(tnb)
            
            batch_x = Img[select]
            batch_y = Lab[select]
            batch_x1 = freq1[select]
            
            rev_batch_y=np.array(conv_mask_gt(batch_y))
            batch_x=np.multiply(batch_x,1.0/mx)
            sess.run(update, feed_dict={input_layer: batch_x, y: rev_batch_y, freqFeat: batch_x1})
            train_loss += sess.run(loss, feed_dict={input_layer: batch_x, y: rev_batch_y, freqFeat: batch_x1})
            
            learn_step +=1  
        train_loss /= train_batches
        vali_loss = 0
        vali_accuracy = 0
        vali_batches = vx.shape[0]//batch_size
        print()
        
        print('vali_batches=',vali_batches)
        for j in range(vali_batches):
#             print('\rj: %d'%j)
            select = next(vnb)
            batch_X = vx[select]
            batch_Y  = vy[select]
            batch_X1 = vx1[select]
            batch_X=np.multiply(batch_X,1.0/mx)
            rev_batch_y=np.array(conv_mask_gt(batch_Y))

            vali_loss += sess.run(loss, feed_dict={input_layer: batch_X, y: rev_batch_y, freqFeat: batch_X1})
            vali_accuracy += sess.run(accuracy, feed_dict={input_layer: batch_X, y: rev_batch_y, freqFeat: batch_X1})
        vali_loss /= vali_batches
        vali_accuracy /= vali_batches
        
        epoch = math.ceil(learn_step / (Lab.shape[0] // batch_size)) 
        print()
        print(datetime.datetime.now( ).strftime('%m-%d %H:%M:%S') + 
                  ' | Step:%6d(%3d/%3d) LR:%.6f TLoss:%.6f VLoss:%f VAcc:%f' % (
                      learn_step, epoch, total_epoch, learn_rate, train_loss, vali_loss, vali_accuracy))
        if learn_step > train_step:
                break
        print('learn_step=',learn_step)
        if learn_step % int(show_freq * train_batches)==0:
            
            
            TP = 0; FP = 0;TN = 0; FN = 0 
            #TP1=0;FP1=0
            num_images=batch_size
            n_chunks=np.shape(tx)[0]//batch_size
            tAcc=np.zeros(n_chunks)

            for chunk in range(0,n_chunks):               
                tx_batch=tx[((chunk)*num_images):((chunk+1)*num_images),...]
                ty_batch=ty[((chunk)*num_images):((chunk+1)*num_images),...]
                tx1_batch=freq4[((chunk)*num_images):((chunk+1)*num_images),...]
                ty_batch=conv_mask_gt(ty_batch)
                tAcc[chunk],y2,p2=sess.run([accuracy,y_actual,y_pred], feed_dict={input_layer: tx_batch, y:ty_batch, freqFeat: tx1_batch})
                a,b,c,d=compute_pos_neg(y2,p2)

                TP+=a; FP+=b;TN+=c; FN+=d
                
            print("TP =",TP)
            print("FP = ",FP)
            print("TN = ",TN)
            print("FN = ",FN)
            
            prec=metrics(TP,FP,TN,FN)            
            test_accuracy=np.mean(tAcc)
            print("prec = ",prec)
            print("test_accuracy = ",test_accuracy)
           
            if prec > 0.45 :
                best_prec = prec
                save_path=saver.save(sess,'../model_shi/final_model_nist.ckpt')
                print ("Best Model Found on NC16...")
                print ( "prec = "+str(prec) + ", acc = "+ str(test_accuracy))

            saver.save(sess, '../model_shi/modelS%d.ckpt' % learn_step)
            
        
    saver.save(sess, '../model_shi/modelS.ckpt')

In [ ]:
show_freq =1
learn_step =0
learn_rate = 0.00003
train_step = 100000
batch_size = 30
with tf.Session(config=config) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    tnb = next_batch(Lab.shape[0], batch_size)
    vnb = next_batch(vx.shape[0], batch_size)
    total_epoch = math.ceil(train_step / (Lab.shape[0] // batch_size))
    print('total_epoch=',total_epoch)
    
    while True:
        
        train_loss = 0
        train_batches = int(show_freq * Lab.shape[0] // batch_size)
        print('train_batches=',train_batches)
        
        for i in range(train_batches):
#             print('\ri: %d'%i)
            sys.stdout.write('\r' + str(i))
            sys.stdout.flush()
#             print('\ri: %d'%i,  end= flush=True)
            select = next(tnb)
            
            batch_x = Img[select]
            batch_y = Lab[select]
            batch_x1 = freq1[select]
            
            rev_batch_y=np.array(conv_mask_gt(batch_y))
            batch_x=np.multiply(batch_x,1.0/mx)
            sess.run(update, feed_dict={input_layer: batch_x, y: rev_batch_y, freqFeat: batch_x1})
            train_loss += sess.run(loss, feed_dict={input_layer: batch_x, y: rev_batch_y, freqFeat: batch_x1})
            
            learn_step +=1  
        train_loss /= train_batches
        vali_loss = 0
        vali_accuracy = 0
        vali_batches = vx.shape[0]//batch_size
        print()
        
        print('vali_batches=',vali_batches)
        for j in range(vali_batches):
#             print('\rj: %d'%j)
            select = next(vnb)
            batch_X = vx[select]
            batch_Y  = vy[select]
            batch_X1 = vx1[select]
            batch_X=np.multiply(batch_X,1.0/mx)
            rev_batch_y=np.array(conv_mask_gt(batch_Y))

            vali_loss += sess.run(loss, feed_dict={input_layer: batch_X, y: rev_batch_y, freqFeat: batch_X1})
            vali_accuracy += sess.run(accuracy, feed_dict={input_layer: batch_X, y: rev_batch_y, freqFeat: batch_X1})
        vali_loss /= vali_batches
        vali_accuracy /= vali_batches
        
        epoch = math.ceil(learn_step / (Lab.shape[0] // batch_size)) 
        print()
        print(datetime.datetime.now( ).strftime('%m-%d %H:%M:%S') + 
                  ' | Step:%6d(%3d/%3d) LR:%.6f TLoss:%.6f VLoss:%f VAcc:%f' % (
                      learn_step, epoch, total_epoch, learn_rate, train_loss, vali_loss, vali_accuracy))
        if learn_step > train_step:
                break
        print('learn_step=',learn_step)
        if learn_step % int(show_freq * train_batches)==0:
            
            
            TP = 0; FP = 0;TN = 0; FN = 0 
            #TP1=0;FP1=0
            num_images=batch_size
            n_chunks=np.shape(tx)[0]//batch_size
            tAcc=np.zeros(n_chunks)

            for chunk in range(0,n_chunks):               
                tx_batch=tx[((chunk)*num_images):((chunk+1)*num_images),...]
                ty_batch=ty[((chunk)*num_images):((chunk+1)*num_images),...]
                tx1_batch=freq4[((chunk)*num_images):((chunk+1)*num_images),...]
                ty_batch=conv_mask_gt(ty_batch)
                tAcc[chunk],y2,p2=sess.run([accuracy,y_actual,y_pred], feed_dict={input_layer: tx_batch, y:ty_batch, freqFeat: tx1_batch})
                a,b,c,d=compute_pos_neg(y2,p2)

                TP+=a; FP+=b;TN+=c; FN+=d
            
            prec=metrics(TP,FP,TN,FN)            
            test_accuracy=np.mean(tAcc)
       
            print(' | TP：%d FP:%d TN:%d FN:%d----prec:%.6f-----Tacc:%.6f' % (TP, FP, TN, FN, prec, test_accuracy))
            
            if prec > 0.30 :
                best_prec = prec
                save_path=saver.save(sess,'../model_s_gaijin1/final_model_nist.ckpt')
                print ("Best Model Found on NC16...")
                print ( "prec = "+str(prec) + ", acc = "+ str(test_accuracy))

            saver.save(sess, '../model_s_gaijin1/modelS%d.ckpt' % learn_step)
            
        
    saver.save(sess, '../model_s_gaijin1/modelS.ckpt')

# model-train

In [ ]:
show_freq =1
learn_step =0
learn_rate = 0.00003
train_step = 100000
batch_size = 30
with tf.Session(config=config) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    tnb = next_batch(Lab.shape[0], batch_size)
    vnb = next_batch(vx.shape[0], batch_size)
    total_epoch = math.ceil(train_step / (Lab.shape[0] // batch_size))
    print('total_epoch=',total_epoch)
    
    while True:
        
        train_loss = 0
        train_batches = int(show_freq * Lab.shape[0] // batch_size)
#         print('train_batches=',train_batches)
        
        for i in range(train_batches):
            sys.stdout.write('\r' + str(i))
            sys.stdout.flush()
            select = next(tnb)
            
            batch_x = Img[select]
            batch_y = Lab[select]
            batch_x1 = freq1[select]

            rev_batch_y=np.array(conv_mask_gt(batch_y))
            batch_x=np.multiply(batch_x,1.0/mx)
            sess.run(update, feed_dict={input_layer: batch_x, y: rev_batch_y, freqFeat: batch_x1})
            train_loss += sess.run(loss, feed_dict={input_layer: batch_x, y: rev_batch_y, freqFeat: batch_x1})
            
            learn_step +=1  
        train_loss /= train_batches
        vali_loss = 0
        vali_accuracy = 0
        vali_batches = vx.shape[0]//batch_size
#         print()
        
#         print('vali_batches=',vali_batches)
        for j in range(vali_batches):
#             print('\rj: %d'%j)
            select = next(vnb)
            batch_X = vx[select]
            batch_Y  = vy[select]
            batch_X1 = vx1[select]
            batch_X=np.multiply(batch_X,1.0/mx)
            rev_batch_y=np.array(conv_mask_gt(batch_Y))

            vali_loss += sess.run(loss, feed_dict={input_layer: batch_X, y: rev_batch_y, freqFeat: batch_X1})
            vali_accuracy += sess.run(accuracy, feed_dict={input_layer: batch_X, y: rev_batch_y, freqFeat: batch_X1})
        vali_loss /= vali_batches
        vali_accuracy /= vali_batches
        
        epoch = math.ceil(learn_step / (Lab.shape[0] // batch_size)) 
       
        print(datetime.datetime.now( ).strftime('%m-%d %H:%M:%S') + 
                  ' | Step:%6d(%3d/%3d) LR:%.6f TLoss:%.6f VLoss:%f VAcc:%f' % (
                      learn_step, epoch, total_epoch, learn_rate, train_loss, vali_loss, vali_accuracy))
        if learn_step > train_step:
                break
        print('learn_step=',learn_step)
        if learn_step % int(show_freq * train_batches)==0:
            
            
            TP = 0; FP = 0;TN = 0; FN = 0 
            #TP1=0;FP1=0
            num_images=batch_size
            n_chunks=np.shape(tx)[0]//batch_size
            tAcc=np.zeros(n_chunks)

            for chunk in range(0,n_chunks):               
                tx_batch=tx[((chunk)*num_images):((chunk+1)*num_images),...]
                ty_batch=ty[((chunk)*num_images):((chunk+1)*num_images),...]
                tx1_batch=freq4[((chunk)*num_images):((chunk+1)*num_images),...]
                ty_batch=conv_mask_gt(ty_batch)
                tAcc[chunk],y2,p2=sess.run([accuracy,y_actual,y_pred], feed_dict={input_layer: tx_batch, y:ty_batch, freqFeat: tx1_batch})
                a,b,c,d=compute_pos_neg(y2,p2)

                TP+=a; FP+=b;TN+=c; FN+=d
            
            prec=metrics(TP,FP,TN,FN)            
            test_accuracy=np.mean(tAcc)
       
            print(' | TP：%d FP:%d TN:%d FN:%d----prec:%.6f-----Tacc:%.6f' % (TP, FP, TN, FN, prec, test_accuracy))
            
            if prec > 0.45 :
                best_prec = prec
                save_path=saver.save(sess,'../model_s/final_model_nist.ckpt')
                print ("Best Model Found on NC16...")
                print ( "prec = "+str(prec) + ", acc = "+ str(test_accuracy))

            saver.save(sess, '../model_s/modelS%d.ckpt' % learn_step)
            
        
    saver.save(sess, '../model_s/modelS.ckpt')

In [ ]:
show_freq =1
learn_step =0
learn_rate = 0.00003
train_step = 1000000
batch_size = 30
with tf.Session(config=config) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    tnb = next_batch(Lab.shape[0], batch_size)
#     vnb = next_batch(vx.shape[0], batch_size)
    total_epoch = math.ceil(train_step / (Lab.shape[0] // batch_size))
    print('total_epoch=',total_epoch)
    
    while True:
        
        train_loss = 0
        train_accuracy = 0
        train_batches = int(show_freq * Lab.shape[0] // batch_size)
#         print('train_batches=',train_batches)
        
        for i in range(train_batches):
#             print('\ri: %d'%i)
            sys.stdout.write('\r' + str(i))
            sys.stdout.flush()
#             print('\ri: %d'%i,  end= flush=True)
            select = next(tnb)
            
            batch_x = Img[select]
            batch_y = Lab[select]
            batch_x1 = freq1[select]
            
            rev_batch_y=np.array(conv_mask_gt(batch_y))
            batch_x=np.multiply(batch_x,1.0/mx)
            sess.run(update, feed_dict={input_layer: batch_x, y: rev_batch_y})
            train_loss += sess.run(loss, feed_dict={input_layer: batch_x, y: rev_batch_y})
            train_accuracy += sess.run(accuracy, feed_dict={input_layer: batch_x, y: rev_batch_y})
            learn_step +=1  
        train_loss /= train_batches
        
        train_accuracy /= train_batches
#         vali_loss = 0
#         vali_accuracy = 0
#         vali_batches = vx.shape[0]//batch_size
        
        
#         print('vali_batches=',vali_batches)
#         for j in range(vali_batches):
# #             print('\rj: %d'%j)
#             select = next(vnb)
#             batch_X = vx[select]
#             batch_Y  = vy[select]
#             batch_X1 = vx1[select]
#             batch_X=np.multiply(batch_X,1.0/mx)
#             rev_batch_y=np.array(conv_mask_gt(batch_Y))

#             vali_loss += sess.run(loss, feed_dict={input_layer: batch_X, y: rev_batch_y, freqFeat: batch_X1})
#             vali_accuracy += sess.run(accuracy, feed_dict={input_layer: batch_X, y: rev_batch_y, freqFeat: batch_X1})
#         vali_loss /= vali_batches
#         vali_accuracy /= vali_batches
        
        epoch = math.ceil(learn_step / (Lab.shape[0] // batch_size)) 
        print()
        
        if learn_step > train_step:
                break
        print('learn_step=',learn_step)
        
        
        
        if learn_step % int(show_freq * train_batches)==0:
            
            
            TP = 0; FP = 0;TN = 0; FN = 0 
            #TP1=0;FP1=0
            num_images=batch_size
            n_chunks=np.shape(tx)[0]//batch_size
            tAcc=np.zeros(n_chunks)
            vali_loss = 0
            for chunk in range(0,n_chunks):               
                tx_batch=tx[((chunk)*num_images):((chunk+1)*num_images),...]
                tx_batch=np.multiply(tx_batch,1.0/mx)
                ty_batch=ty[((chunk)*num_images):((chunk+1)*num_images),...]
                tx1_batch=freq4[((chunk)*num_images):((chunk+1)*num_images),...]
                ty_batch=conv_mask_gt(ty_batch)
                
                vali_loss += sess.run(loss, feed_dict={input_layer: tx_batch, y:ty_batch})
        
                
                tAcc[chunk],y2,p2=sess.run([accuracy,y_actual,y_pred], feed_dict={input_layer: tx_batch, y:ty_batch})
                a,b,c,d=compute_pos_neg(y2,p2)

                TP+=a; FP+=b;TN+=c; FN+=d
            vali_loss /= n_chunks
            prec=metrics(TP,FP,TN,FN)            
            test_accuracy=np.mean(tAcc)
            print(datetime.datetime.now( ).strftime('%m-%d %H:%M:%S') + 
                  ' | Step:%6d(%3d/%3d) LR:%.6f TrainLoss:%.6f ValiLoss:%f TrainAcc:%.6f' % (
                      learn_step, epoch, total_epoch, learn_rate, train_loss, vali_loss,train_accuracy))

            print(' | TP：%d FP:%d TN:%d FN:%d----prec:%.6f-----Tacc:%.6f' % (TP, FP, TN, FN, prec, test_accuracy))
            
            if prec > 0.55 :
                best_prec = prec
                save_path=saver.save(sess,'../model_s/final_model_nist.ckpt')
                print ("Best Model Found on NC16...")
                print ( "prec = "+str(prec) + ", acc = "+ str(test_accuracy))

            saver.save(sess, '../model_s/modelS%d.ckpt' % learn_step)
            
        
    saver.save(sess, '../model_s/modelS.ckpt')